# Otimização de Programação de Férias usando MILP

## Objetivo

Esta PoC utiliza **Programação Linear Inteira (MILP)** para otimizar a alocação de dias de férias, maximizando o período contínuo de descanso considerando:

- Dias úteis (segunda a sexta)
- Fins de semana (sábado e domingo)
- Feriados nacionais
- Feriados "imprensados" (pontes)

## Premissas

- Ano fixo: 2026
- Apenas um bloco contínuo de férias
- Dias de férias consomem apenas dias úteis
- Fins de semana e feriados não consomem dias de férias

## 1. Importações e Configurações

In [1]:
from datetime import datetime, timedelta
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus
import pandas as pd

# Ano fixo
ANO = 2026

## 2. Definição de Feriados Nacionais (2026)

In [2]:
# Feriados nacionais fixos e móveis para 2026 (Brasil)
# Fonte: calendário oficial brasileiro

FERIADOS_FIXOS = [
    datetime(2026, 1, 1),   # Ano Novo
    datetime(2026, 4, 21),  # Tiradentes
    datetime(2026, 5, 1),   # Dia do Trabalhador
    datetime(2026, 9, 7),   # Independência
    datetime(2026, 10, 12), # Nossa Senhora Aparecida
    datetime(2026, 11, 2),  # Finados
    datetime(2026, 11, 15), # Proclamação da República
    datetime(2026, 12, 25), # Natal
]

# Feriados móveis (calculados para 2026)
# Carnaval: 16 e 17 de fevereiro de 2026
# Sexta-feira Santa: 3 de abril de 2026
# Páscoa: 5 de abril de 2026

FERIADOS_MOVEIS = [
    datetime(2026, 2, 16),  # Carnaval (segunda)
    datetime(2026, 2, 17),  # Carnaval (terça)
    datetime(2026, 4, 3),   # Sexta-feira Santa
]

# Todos os feriados
FERIADOS = FERIADOS_FIXOS + FERIADOS_MOVEIS

print(f"Total de feriados em {ANO}: {len(FERIADOS)}")
for feriado in sorted(FERIADOS):
    print(f"  {feriado.strftime('%d/%m/%Y')} - {feriado.strftime('%A')}")

Total de feriados em 2026: 11
  01/01/2026 - Thursday
  16/02/2026 - Monday
  17/02/2026 - Tuesday
  03/04/2026 - Friday
  21/04/2026 - Tuesday
  01/05/2026 - Friday
  07/09/2026 - Monday
  12/10/2026 - Monday
  02/11/2026 - Monday
  15/11/2026 - Sunday
  25/12/2026 - Friday


## 3. Geração do Calendário e Classificação de Dias

In [3]:
def gerar_calendario(ano):
    """
    Gera o calendário completo do ano e classifica cada dia.
    
    Retorna:
    - Lista de datas do ano
    - Dicionário com classificação de cada data
    """
    inicio_ano = datetime(ano, 1, 1)
    fim_ano = datetime(ano, 12, 31)
    
    calendario = []
    classificacao = {}
    
    data_atual = inicio_ano
    while data_atual <= fim_ano:
        calendario.append(data_atual)
        
        # Classificação do dia
        dia_semana = data_atual.weekday()  # 0=segunda, 6=domingo
        
        if data_atual.date() in [f.date() for f in FERIADOS]:
            classificacao[data_atual] = 'feriado'
        elif dia_semana >= 5:  # Sábado (5) ou Domingo (6)
            classificacao[data_atual] = 'fim_semana'
        else:
            classificacao[data_atual] = 'dia_util'
        
        data_atual += timedelta(days=1)
    
    return calendario, classificacao

# Gerar calendário
calendario, classificacao = gerar_calendario(ANO)

# Estatísticas
total_dias = len(calendario)
dias_uteis = sum(1 for c in classificacao.values() if c == 'dia_util')
fins_semana = sum(1 for c in classificacao.values() if c == 'fim_semana')
feriados_count = sum(1 for c in classificacao.values() if c == 'feriado')

print(f"\nEstatísticas do calendário {ANO}:")
print(f"  Total de dias: {total_dias}")
print(f"  Dias úteis: {dias_uteis}")
print(f"  Fins de semana: {fins_semana}")
print(f"  Feriados: {feriados_count}")


Estatísticas do calendário 2026:
  Total de dias: 365
  Dias úteis: 251
  Fins de semana: 103
  Feriados: 11


## 3.5. Função Auxiliar para Cálculo de Período de Descanso

In [4]:
def calcular_periodo_descanso(primeiro_dia_util, ultimo_dia_util, calendario, classificacao):
    """
    Calcula o número total de dias de descanso (dias úteis + fins de semana + feriados)
    entre o primeiro e último dia útil, incluindo expansão para fins de semana/feriados adjacentes.
    """
    # Expandir para trás
    data_inicio = primeiro_dia_util
    while True:
        data_anterior = data_inicio - timedelta(days=1)
        if data_anterior.year != ANO or classificacao.get(data_anterior) == 'dia_util':
            break
        data_inicio = data_anterior
    
    # Expandir para frente
    data_fim = ultimo_dia_util
    while True:
        data_proxima = data_fim + timedelta(days=1)
        if data_proxima.year != ANO or classificacao.get(data_proxima) == 'dia_util':
            break
        data_fim = data_proxima
    
    # Contar todos os dias no período
    periodo = []
    data_atual = data_inicio
    while data_atual <= data_fim:
        periodo.append(data_atual)
        data_atual += timedelta(days=1)
    
    return len(periodo)

print("Função auxiliar definida.")

Função auxiliar definida.


## 4. Modelagem MILP

### Variáveis de Decisão

Para cada dia útil do ano, definimos uma variável binária:
- `x[i] = 1` se o dia útil `i` é usado como férias
- `x[i] = 0` caso contrário

### Restrições

1. **Número exato de dias de férias**: A soma das variáveis deve ser igual ao número solicitado
2. **Continuidade**: As férias devem formar um bloco contínuo

### Função Objetivo

Maximizar o tamanho do intervalo contínuo de descanso (dias de férias + fins de semana + feriados consecutivos)

In [5]:
def resolver_otimizacao_férias(dias_ferias_solicitados, calendario, classificacao):
    """
    Resolve o problema de otimização de férias usando MILP.
    
    Estratégia: Para cada possível início de bloco, pré-calculamos o período de descanso total.
    O modelo escolhe o início que maximiza esse período.
    
    Args:
        dias_ferias_solicitados: Número de dias úteis de férias desejados
        calendario: Lista de todas as datas do ano
        classificacao: Dicionário com classificação de cada data
    
    Returns:
        Solução do modelo (variáveis, status, etc.)
    """
    
    # Filtrar apenas dias úteis
    dias_uteis = [data for data in calendario if classificacao[data] == 'dia_util']
    n_dias_uteis = len(dias_uteis)
    
    # Pré-calcular período de descanso para cada possível início de bloco
    # inicio_possivel[i] = período de descanso total se o bloco começar no dia útil i
    inicio_possivel = {}
    for i in range(n_dias_uteis - dias_ferias_solicitados + 1):
        primeiro = dias_uteis[i]
        ultimo = dias_uteis[i + dias_ferias_solicitados - 1]
        periodo_descanso = calcular_periodo_descanso(primeiro, ultimo, calendario, classificacao)
        inicio_possivel[i] = periodo_descanso
    
    # Criar modelo
    modelo = LpProblem("Otimizacao_Férias", LpMaximize)
    
    # Variável de decisão: y[i] = 1 se o bloco de férias começa no dia útil i
    y = {i: LpVariable(f"y_{i}", cat='Binary') for i in inicio_possivel.keys()}
    
    # Variáveis de decisão auxiliares: x[i] = 1 se o dia útil i é usado como férias
    x = {i: LpVariable(f"x_{i}", cat='Binary') for i in range(n_dias_uteis)}
    
    # Restrição 1: Apenas um início de bloco
    modelo += lpSum([y[i] for i in inicio_possivel.keys()]) == 1
    
    # Restrição 2: Se y[i] = 1, então x[i] até x[i+dias_ferias_solicitados-1] devem ser 1
    for i in inicio_possivel.keys():
        for j in range(dias_ferias_solicitados):
            modelo += x[i + j] >= y[i]
    
    # Restrição 3: Se y[i] = 1, então x[j] = 0 para j < i ou j >= i + dias_ferias_solicitados
    for i in inicio_possivel.keys():
        for j in range(n_dias_uteis):
            if j < i or j >= i + dias_ferias_solicitados:
                modelo += x[j] <= 1 - y[i]
    
    # Função objetivo: Maximizar o período de descanso total
    # Usamos os valores pré-calculados
    modelo += lpSum([inicio_possivel[i] * y[i] for i in inicio_possivel.keys()])
    
    # Resolver
    modelo.solve()
    
    return modelo, x, y, dias_uteis

print("Função de otimização definida.")

Função de otimização definida.


## 5. Resolução e Pós-processamento

In [6]:
def processar_solucao(modelo, x, y, dias_uteis, calendario, classificacao):
    """
    Processa a solução do modelo e calcula estatísticas.
    """
    if LpStatus[modelo.status] != 'Optimal':
        return None, f"Status do solver: {LpStatus[modelo.status]}"
    
    # Identificar qual início foi selecionado
    inicio_selecionado = None
    for i in y.keys():
        if y[i].varValue == 1:
            inicio_selecionado = i
            break
    
    if inicio_selecionado is None:
        return None, "Nenhum início de bloco selecionado"
    
    # Identificar quais dias úteis foram selecionados
    dias_selecionados = []
    for i in range(len(dias_uteis)):
        if x[i].varValue == 1:
            dias_selecionados.append(dias_uteis[i])
    
    if not dias_selecionados:
        return None, "Nenhum dia selecionado"
    
    # Primeiro e último dia útil de férias
    primeiro_dia_util = min(dias_selecionados)
    ultimo_dia_util = max(dias_selecionados)
    
    # Calcular intervalo completo de descanso (incluindo fins de semana e feriados)
    # Expandir para trás até encontrar um dia útil não-feriado
    data_inicio = primeiro_dia_util
    while True:
        data_anterior = data_inicio - timedelta(days=1)
        if data_anterior.year != ANO:
            break
        if classificacao.get(data_anterior) in ['fim_semana', 'feriado']:
            data_inicio = data_anterior
        else:
            break
    
    # Expandir para frente
    data_fim = ultimo_dia_util
    while True:
        data_proxima = data_fim + timedelta(days=1)
        if data_proxima.year != ANO:
            break
        if classificacao.get(data_proxima) in ['fim_semana', 'feriado']:
            data_fim = data_proxima
        else:
            break
    
    # Construir período completo
    periodo_completo = []
    data_atual = data_inicio
    while data_atual <= data_fim:
        periodo_completo.append(data_atual)
        data_atual += timedelta(days=1)
    
    primeiro_dia_corrido = min(periodo_completo)
    ultimo_dia_corrido = max(periodo_completo)
    
    # Contar dias de descanso total
    dias_descanso_total = len(periodo_completo)
    
    resultado = {
        'inicio': primeiro_dia_corrido,
        'fim': ultimo_dia_corrido,
        'dias_ferias_usados': len(dias_selecionados),
        'dias_descanso_total': dias_descanso_total,
        'dias_uteis_selecionados': dias_selecionados,
        'periodo_completo': periodo_completo
    }
    
    return resultado, None

print("Função de processamento definida.")

Função de processamento definida.


## 6. Interface do Usuário e Execução

In [7]:
def otimizar_ferias(dias_ferias_solicitados):
    """
    Função principal que executa toda a otimização.
    """
    print(f"\n{'='*60}")
    print(f"Otimizando {dias_ferias_solicitados} dias de férias para {ANO}")
    print(f"{'='*60}\n")
    
    # Resolver modelo
    modelo, x, y, dias_uteis = resolver_otimizacao_férias(
        dias_ferias_solicitados, calendario, classificacao
    )
    
    # Processar solução
    resultado, erro = processar_solucao(modelo, x, y, dias_uteis, calendario, classificacao)
    
    if erro:
        print(f"Erro: {erro}")
        return None
    
    # Exibir resultado
    print("\n" + "="*60)
    print("SUGESTÃO DE FÉRIAS")
    print("="*60)
    print(f"Início: {resultado['inicio'].strftime('%d/%m/%Y')}")
    print(f"Fim: {resultado['fim'].strftime('%d/%m/%Y')}")
    print(f"Dias de férias usados: {resultado['dias_ferias_usados']}")
    print(f"Dias totais de descanso: {resultado['dias_descanso_total']}")
    print("\n" + "="*60)
    print("EXPLICAÇÃO")
    print("="*60)
    
    # Calcular estatísticas do período
    periodo = resultado['periodo_completo']
    fins_semana_periodo = sum(1 for d in periodo if classificacao.get(d) == 'fim_semana')
    feriados_periodo = sum(1 for d in periodo if classificacao.get(d) == 'feriado')
    
    explicacao = f"""
O período selecionado aproveita:
- {fins_semana_periodo} dias de fim de semana
- {feriados_periodo} feriados

Isso resulta em {resultado['dias_descanso_total']} dias consecutivos de descanso
usando apenas {resultado['dias_ferias_usados']} dias úteis de férias.

O modelo MILP garantiu:
✓ Continuidade do bloco de férias
✓ Maximização do período de descanso
✓ Aproveitamento ótimo de fins de semana e feriados
    """
    
    print(explicacao)
    
    return resultado

# Exemplo de uso
dias_solicitados = 15
resultado = otimizar_ferias(dias_solicitados)


Otimizando 15 dias de férias para 2026


SUGESTÃO DE FÉRIAS
Início: 13/02/2026
Fim: 09/03/2026
Dias de férias usados: 15
Dias totais de descanso: 25

EXPLICAÇÃO

O período selecionado aproveita:
- 8 dias de fim de semana
- 2 feriados

Isso resulta em 25 dias consecutivos de descanso
usando apenas 15 dias úteis de férias.

O modelo MILP garantiu:
✓ Continuidade do bloco de férias
✓ Maximização do período de descanso
✓ Aproveitamento ótimo de fins de semana e feriados
    


## 7. Teste com Diferentes Valores

In [8]:
# Testar com diferentes quantidades de dias
for dias in [10, 15, 20, 30]:
    print(f"\n\n{'#'*60}")
    print(f"Teste com {dias} dias de férias")
    print(f"{'#'*60}")
    resultado = otimizar_ferias(dias)
    if resultado:
        print(f"\n✓ Solução encontrada!")
    else:
        print(f"\n✗ Não foi possível encontrar solução.")



############################################################
Teste com 10 dias de férias
############################################################

Otimizando 10 dias de férias para 2026


SUGESTÃO DE FÉRIAS
Início: 05/02/2026
Fim: 22/02/2026
Dias de férias usados: 10
Dias totais de descanso: 18

EXPLICAÇÃO

O período selecionado aproveita:
- 6 dias de fim de semana
- 2 feriados

Isso resulta em 18 dias consecutivos de descanso
usando apenas 10 dias úteis de férias.

O modelo MILP garantiu:
✓ Continuidade do bloco de férias
✓ Maximização do período de descanso
✓ Aproveitamento ótimo de fins de semana e feriados
    

✓ Solução encontrada!


############################################################
Teste com 15 dias de férias
############################################################

Otimizando 15 dias de férias para 2026


SUGESTÃO DE FÉRIAS
Início: 13/02/2026
Fim: 09/03/2026
Dias de férias usados: 15
Dias totais de descanso: 25

EXPLICAÇÃO

O período selecionado aproveita:
